# Sentiment Analysis of Amazon Reviews

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import nltk

In [ ]:
# Import data
df = pd.read_csv('../data/amazon_reviews.csv')